%%writefile creation_db.hql

DROP DATABASE IF EXISTS demodb CASCADE;

%%writefile -a creation_db.hql
CREATE DATABASE demodb LOCATION '/user/jovyan/somemetastore';

! hive -f creation_db.hql

In [7]:
%%writefile external_table.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;
DROP TABLE IF EXISTS posts_sample;
CREATE EXTERNAL TABLE posts_sample_external (
id int,
posttypeid int,
creationdate string,
tags string,
owneruserid int,
parentid int,
score int,
favoritecount int
)
ROW FORMAT
SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
"input.regex" = '\<row (?=.*\\bId=\"(\\d+))(?=.*\\bPostTypeId=\"(\\d+))(?=.*\\bCreationDate=\"([0-9-T:.]+)\")(?=.*\\bTags=\"(\\S+)\")?(?=.*\bOwnerUserId=\"(\\d+))?(?=.*\\bParentId=\"(\\d+))?(?=.*\\bScore=\"(\\d\+))(?=.*\\bFavoriteCount =\"(\\d+))?.*$'
)
LOCATION '/data/stackexchange1000/posts/';

CREATE TABLE posts_sample (
id int,
posttypeid int,
creationdate string,
tags string,
owneruserid int,
parentid int,
score int,
favoritecount int
)
PARTITIONED BY (year STRING,month STRING)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE;



Overwriting external_table.hql


In [8]:
%%writefile -a external_table.hql
SET hive.exec.dynamic.partition = true;
set hive.exec.dynamic.partition.mode=nonstrict;
set hive.exec.max.dynamic.partitions=2000;
set hive.exec.max.dynamic.partitions.pernode=1000;
INSERT OVERWRITE TABLE posts_sample partition(year,month)
Select 
id ,
posttypeid ,
creationdate ,
tags ,
owneruserid ,
parentid ,
score ,
favoritecount,
substr(creationdate,1,4) as year,
substr(creationdate,1,7) as month
from posts_sample_external ;
with 
B as
(
    select year,month,cnt,row_number() over(order by month asc) as rn from A 
),
A as
(select year,month,count(*) as cnt from posts_sample group by year,month )
select year,month,cnt from B where rn=3;

Appending to external_table.hql


In [9]:
! hive -f external_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.088 seconds
OK
Time taken: 1.588 seconds
OK
Time taken: 3.149 seconds
OK
Time taken: 0.698 seconds
OK
Time taken: 0.291 seconds
Query ID = jovyan_20180228054242_c63011ea-fd31-4966-87b5-9bbcb58fed47
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1519792833153_0011, Tracking URL = http://f860572437fc:8088/proxy/application_1519792833153_0011/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1519792833153_0011
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2018-02-28 05:42:49,523 Stage-1 map = 0%,  reduce = 0%
2018-02-28 05:43:09,031 Stage-1 map = 24%,  reduce = 0%, Cumulati

Partition demodb.posts_sample{year=2011, month=2011-01} stats: [numFiles=1, numRows=273, totalSize=18392, rawDataSize=18119]
Partition demodb.posts_sample{year=2011, month=2011-02} stats: [numFiles=1, numRows=269, totalSize=18161, rawDataSize=17892]
Partition demodb.posts_sample{year=2011, month=2011-03} stats: [numFiles=1, numRows=346, totalSize=22793, rawDataSize=22447]
Partition demodb.posts_sample{year=2011, month=2011-04} stats: [numFiles=1, numRows=318, totalSize=20739, rawDataSize=20421]
Partition demodb.posts_sample{year=2011, month=2011-05} stats: [numFiles=1, numRows=324, totalSize=21827, rawDataSize=21503]
Partition demodb.posts_sample{year=2011, month=2011-06} stats: [numFiles=1, numRows=339, totalSize=22503, rawDataSize=22164]
Partition demodb.posts_sample{year=2011, month=2011-07} stats: [numFiles=1, numRows=329, totalSize=21881, rawDataSize=21552]
Partition demodb.posts_sample{year=2011, month=2011-08} stats: [numFiles=1, numRows=353, totalSize=23490, rawDataSize=23137]


Partition demodb.posts_sample{year=2016, month=2016-10} stats: [numFiles=1, numRows=517, totalSize=37828, rawDataSize=37311]
Partition demodb.posts_sample{year=2016, month=2016-11} stats: [numFiles=1, numRows=509, totalSize=37731, rawDataSize=37222]
Partition demodb.posts_sample{year=2016, month=2016-12} stats: [numFiles=1, numRows=177, totalSize=12625, rawDataSize=12448]
Partition demodb.posts_sample{year=__HIVE_DEFAULT_PARTITION__, month=__HIVE_DEFAULT_PARTITION__} stats: [numFiles=1, numRows=1747, totalSize=41928, rawDataSize=40181]
MapReduce Jobs Launched: 
Stage-Stage-1: Map: 1   Cumulative CPU: 49.65 sec   HDFS Read: 60004585 HDFS Write: 2664244 SUCCESS
Total MapReduce CPU Time Spent: 49 seconds 650 msec
OK
Time taken: 93.177 seconds
Query ID = jovyan_20180228054444_014cf5cc-0ec6-4bc1-a55e-6630ae072514
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set 